# DIS21a 1 | Big Data | Exam WS 2022/23 | Group J (10)

## Participants

-   [Markus Hardtke](https://elearning.iws.th-koeln.de/moodle/user/profile.php?id=2705)
    -   Matriculation Number: 1234567
-   [Furkan Erdogan](https://elearning.iws.th-koeln.de/moodle/user/profile.php?id=2688)
    -   Matriculation Number: 1234567
-   [Jannik Loose](https://elearning.iws.th-koeln.de/moodle/user/profile.php?id=2687)
    -   Matriculation Number: 1234567
-   [Gilles Romer](https://elearning.iws.th-koeln.de/moodle/user/profile.php?id=2681)
    -   Matriculation Number: 11139919

---

## Project Setup

*This Setup was made for Windows users and will differ if you are using a different OS.    
The recommended Python version used to run this notebook is 3.9.13*

1. Virtual environment

    *This step is required so you can install the required packages without affecting or filling up your global Python installation.*

    ```bash
    python3 -m venv .venv
    source .venv/Scripts/activate
    pip install -r requirements.txt
    ```

2. Git

    *This step is required so unnecessary data from the Notebook is not pushed to the repository. This will automatically remove all metadata and execution counts from the Notebook as soon as you stage your file.*

    1. Download JQ (flexible command-line JSON processor) from [here](https://github.com/stedolan/jq/releases/download/jq-1.6/jq-win64.exe)
    2. Create the following Folder: `C:\Program Files\jq`
    3. Add the following line to your PATH environment variable: `C:\Program Files\jq`
    4. Rename the downloaded JQ file to `jq.exe` and move it to the previously created folder
    5. Add the following lines to your .gitconfig file (usually found in C:\Users\YOUR_USERNAME\.gitconfig)

        ```bash
        [core]
            attributesfile = ~/.gitattributes_global
        [filter "nbstrip_meta"]
            clean = "jq --indent 4 \
                    '(.cells[] | select(has(\"execution_count\")) | .execution_count) = null  \
                    | .metadata = {\"language_info\": {\"name\": \"python\", \"pygments_lexer\": \"ipython3\"}} \
                    | .cells[].metadata = {} \
                    '"
            smudge = cat
            required = true
        [filter "nbstrip_full"]
            clean = "jq --indent 4 \
                    '(.cells[] | select(has(\"outputs\")) | .outputs) = []  \
                    | (.cells[] | select(has(\"execution_count\")) | .execution_count) = null  \
                    | .metadata = {\"language_info\": {\"name\": \"python\", \"pygments_lexer\": \"ipython3\"}} \
                    | .cells[].metadata = {} \
                    '"
            smudge = cat
            required = true
        ```

    6. Create a `.gitattributes_global` file at the same location as your `.gitconfig` file and add the following lines:

        ```bash
        *.ipynb filter=nbstrip_meta
        ```

3. Weights and Biases

    *Weights and Biases is a tool that helps you track your experiments and visualize your results. It is also used to run hyperparameter sweeps.*  
    
    Make sure to create an account on [Weights and Biases](https://wandb.ai/site) and accept the invitation to the Team.  
    Furthermore you will have to add your [API key](https://wandb.ai/settings#dangerzone) to the [scrts.py](./scrts.py) file under `wandb_api_key`.

## Optional Setup

1. Tensorflow GPU support  

    *<span style="color:orange">This step is optional and only required if you want to use a GPU for training.</span>*  
   Follow this setup to activate tensorflow GPU support (Make sure to install the mentioned versions rather than the latest one!): [https://www.tensorflow.org/install/gpu](https://www.tensorflow.org/install/gpu)  

---


## Environment Setup

### Path's

In [ ]:
# Importing the necessary libraries
import os
import sys

# Setting up the path to the root of the repository
path = os.getcwd()
notebookpath = os.path.join(path, 'Exam_WS2223.ipynb')

datapath = os.path.join(path, 'data')
data_testpath = os.path.join(datapath, 'test')
data_trainpath = os.path.join(datapath, 'train')
data_valpath = os.path.join(datapath, 'pred')

### Tensorflow version and GPU availability

In [ ]:
# Importing the libraries
import tensorflow as tf
from tensorflow import keras

# Checking versions and gpu availability
print(f"Tensorflow version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Tensorflow version: 2.10.0
Keras Version: 2.10.0
GPU is available


### Wandb

In [ ]:
# Importing the libraries
import wandb
import scrts
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
import logging
import absl.logging
import os

# Removing all unnecessary logs
absl.logging.set_verbosity(absl.logging.ERROR)
logger = logging.getLogger("wandb")
logger.setLevel(logging.ERROR)
os.environ['WANDB_SILENT'] = 'true'
os.environ['WANDB_CONSOLE'] = 'off'
os.environ['WANDB_NOTEBOOK_NAME'] = notebookpath
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

if scrts.wandb_api_key is None:
    print("Please enter your wandb API key in scrts.py")
else:
    wandb.login(key=scrts.wandb_api_key)
wandb.init(project="Exam", entity="dis21a1_ws22-21_gruppe10")